# Memory
Memory allows you to bring previous messages, instructions and responses into the context of a new prompt. This is particularly useful for chatbots and conversation chains.

We can start by initiating the LLM object with a built-in OpenAI chat model.

In [ ]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    openai_api_key="KEY-ABC-DEF",
    model_name='gpt-3.5-turbo'
)

Some orchestration frameworks (like LangChain) have callback handlers to monitor and log LLM tasks. Here we can monitor how many tokens have been submitted.

In [ ]:
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f"Spent a total of {cb.total_tokens} tokens")
    
    return result

We can then define the conversation chain with a memory argument. In this case we are using Summary Memory, which is useful for keeping token counts down on longer conversations. Summary memory makes use of an LLM to create the summary, so that should also be considered in the final token count.

In [ ]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationSummaryMemory

conversation_sum = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=llm)
)

print(conversation_sum.memory.prompt.template)

In the following prompts, watch how the conversation history element changes in the LLM response.

In [ ]:
prompt = "Tell me something about weather in 2018."
conversation_sum(prompt)

In [ ]:
prompt = "Were there any particularly long lasting storms in 2018?"
conversation_sum(prompt)

In [ ]:
prompt = "What were we just talking about?"
conversation_sum(prompt)

In [ ]:
print(conversation_sum.memory.chat_memory.messages)

For applications with recurring users, or long lasting conversations, a database can be used to persist conversation histories. Here we will store each history entry in a seperate row, for simple access with SQL.

In [ ]:
import intersystems_iris.dbapi._DBAPI as iris

conn = iris.connect(hostname='localhost', 
                    port=51972, 
                    namespace='USER',
                    username='SuperUser', 
                    password='SYS')

# Create a table to store message prompts and responses.
cursor = conn.cursor()
cursor.execute("""
        CREATE TABLE RAG_Application.ConversationHistory (
            Username VARCHAR(100),
            ConversationTitle VARCHAR(250),
            HumanMessage VARCHAR(10000),
            AIMessage VARCHAR(10000)
        )
    """)
cursor.close()

Now we can store the conversation messages in a relational table, just using SQL:

In [ ]:
username = input("Enter your username")
chat_history = conversation_sum.memory.chat_memory.messages

cursor = conn.cursor()

for i in range(0,len(chat_history),2):
    conversation_title = "This title"
    human_message = chat_history[i].content
    ai_message = chat_history[i+1].content
    cursor.execute(f"""INSERT INTO RAG_Application.ConversationHistory (Username, ConversationTitle, HumanMessage, AIMessage)
                   VALUES ('{username}', '{conversation_title}', '{human_message}', '{ai_message}')""")

cursor.close()

Retrieve the persisted conversation:

In [ ]:
cursor = conn.cursor()
cursor.execute(f"""SELECT HumanMessage, AIMessage FROM RAG_Application.ConversationHistory 
               WHERE Username = '{username}' AND ConversationTitle = '{conversation_title}'""")

for row in cursor.fetchall():
    print(row)

cursor.close()